# Compte rendu projet numérique de mathématiques

## Question 1

Soit l'ensemble de niveau c est vide, soit l'ensemble de niveau c de f est borné (démo avec limite) et fermé (image réciproque singleton par f continue) si non vide

## Question 2

C'est l'aire algébrique du parallélépipède généré par les vecteurs (d1f(x0),d2f(x0)) et (x10-x1,x20-x2) du plan, divisée par le norme du gradient de f en x0

## Question 3

Théorème des fonctions implicites